In [1]:
# %matplotlib notebook

In [2]:
# 1) Wipe out all Python variables
%reset -f
# 2) Force Python’s garbage collector to run
import gc
gc.collect()
# 3) If you’re using PyTorch + CUDA, free any lingering GPU memory
import torch
torch.cuda.empty_cache()

import math
import pandas as pd
from pandas import Timestamp
import numpy as np

import glob
import os
import datetime as dt
from datetime import datetime
import optuna
import json

import matplotlib.pyplot as plt
from IPython.display import display, clear_output, update_display

import stockanalibs

pd.set_option('display.max_columns', None)


ModuleNotFoundError: No module named 'optuna'

In [ ]:
ticker = stockanalibs.ticker
source_folder = "dfs training" 
results_folder = "optuna results"              

n_trials = 100

path = os.path.join(source_folder, f"{ticker}_base.csv")

df = pd.read_csv(path, index_col=0, parse_dates=["datetime"])

reference_gain = stockanalibs.compute_reference_gain(df)

print('reference_gain', reference_gain)
df

In [ ]:
def optimiz_function(df,
                    min_prof_thr, # percent of minimum profit to define a potential trade
                    max_down_prop, # float (percent/100) of maximum allowed drop of a potential trade
                    gain_tightening_factor, # as gain grows, tighten the stop 'max_down_prop' by this factor.
                    merging_retracement_thr, # intermediate retracement, relative to the first trade's full range
                    merging_time_gap_thr, # time gap between trades, relative to the first and second trade durations
                    smooth_win_sig, # smoothing window of the signal used for the identification of the final trades 
                    pre_entry_decay, # pre-trade decay of the final trades' raw signal
                    buy_threshold, # float (percent/100) threshold of the smoothed signal to trigger the final trade
                    trailing_stop_thresh # percent of the trailing stop loss of the final trade
                    ):
    
    # First, adjust the input DataFrame's timestamps.
    df_prep = stockanalibs.prepare_interpolate_data(df=df, 
                                                    regular_start=stockanalibs.regular_start, 
                                                    regular_end=stockanalibs.regular_end)

    sim_results = stockanalibs.run_trading_pipeline(df_prep=df_prep, 
                                                  reference_gain=reference_gain,
                                                  min_prof_thr=min_prof_thr, 
                                                  max_down_prop=max_down_prop, 
                                                  gain_tightening_factor=gain_tightening_factor, 
                                                  smooth_win_sig=smooth_win_sig, 
                                                  pre_entry_decay=pre_entry_decay,
                                                  buy_threshold=buy_threshold, 
                                                  trailing_stop_thresh=trailing_stop_thresh, 
                                                  merging_retracement_thr=merging_retracement_thr, 
                                                  merging_time_gap_thr=merging_time_gap_thr)
    
    # Build a dictionary mapping each day to its strategy improvement.
    improvements = {}
    for day_key, sim_results in results_by_day.items():
        improvements[day_key.strftime('%Y-%m-%d')] = sim_results[2]['Strategy Improvement (%)']

    avg_improvement = sum(improvements.values()) / len(improvements) 

    # Return the average improvement and the dictionary with day-specific improvements.
    return avg_improvement, improvements


In [ ]:

# === Objective Function ===
def objective(trial):
    # Suggest parameters to test.
    params = {
        "min_prof_thr": trial.suggest_float("min_prof_thr", 0.1, 0.4),
        "max_down_prop": trial.suggest_float("max_down_prop", 0.2, 0.7),
        "gain_tightening_factor": trial.suggest_float("gain_tightening_factor", 0.1, 0.3),
        "merging_retracement_thr": trial.suggest_float("merging_retracement_thr", 0.7, 1),
        "merging_time_gap_thr": trial.suggest_float("merging_time_gap_thr", 0.6, 0.9),
        "smooth_win_sig": trial.suggest_int("smooth_win_sig", 5, 60),
        "pre_entry_decay": trial.suggest_float("pre_entry_decay", 0.001, 0.01),
        "buy_threshold": trial.suggest_float("buy_threshold", 0.05, 0.5),
        "trailing_stop_thresh": trial.suggest_float("trailing_stop_thresh", 0.1, 0.5),
    }

    # Run your strategy simulation with the current set of parameters.
    avg_improvement, improvements_dict = optimiz_function(
        df=df,
        min_prof_thr=params["min_prof_thr"],
        max_down_prop=params["max_down_prop"],
        gain_tightening_factor=params["gain_tightening_factor"],
        merging_retracement_thr=params["merging_retracement_thr"],
        merging_time_gap_thr=params["merging_time_gap_thr"],
        smooth_win_sig=params["smooth_win_sig"],
        pre_entry_decay=params["pre_entry_decay"],
        buy_threshold=params["buy_threshold"],
        trailing_stop_thresh=params["trailing_stop_thresh"]
    )
    
    return avg_improvement



In [ ]:
# ----------------------------------------------------------
# create ONE figure
# ----------------------------------------------------------
fig, ax = plt.subplots(figsize=(7, 3))
line,   = ax.plot([], [], "bo-")
ax.set(xlabel="Trial #", ylabel="Objective",
       title="Optuna optimisation progress", xlim=(0, 1), ylim=(0, 1))
ax.grid(True)

handle = display(fig, display_id=True)   # show once and keep handle
plt.close(fig)                           # <── prevents the duplicate static copy

# ----------------------------------------------------------
# callback
# ----------------------------------------------------------
def live_plot_callback(study, trial):
    xs = [t.number   for t in study.trials]
    ys = [t.value    for t in study.trials]

    line.set_data(xs, ys)
    ax.set_xlim(-1, len(xs))
    ax.set_ylim(min(ys) * 0.95, max(ys) * 1.15)

    update_display(fig, display_id=handle.display_id)   # refresh only this figure




In [ ]:
# === Create and Run the Study ===
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=n_trials, callbacks=[live_plot_callback])

# === Print Final Results ===
print("Best Parameters:", study.best_params)
print("Best Average Improvement:", study.best_value)


# ------------------------------------------------------------------
# Build a dynamic file-name:  <results_folder>/<TICKER>_<YYYYMMDD>-<YYYYMMDD>_optuna.json
# ------------------------------------------------------------------

start_date  = df.index.min().strftime("%Y%m%d")  # ❹ first date in the DataFrame
end_date    = df.index.max().strftime("%Y%m%d")  # ❺ last  date in the DataFrame

file_name = f"{ticker}_{start_date}-{end_date}_optuna_results.json"
file_path = os.path.join(results_folder, file_name)

# ------------------------------------------------------------------
# Dump study results
# ------------------------------------------------------------------
with open(file_path, "w") as f:
    json.dump(
        {
            "best_params": study.best_params,
            "best_value" : study.best_value,
            "trials": [
                {
                    "number" : t.number,
                    "value"  : t.value,
                    "params" : t.params,
                    "state"  : t.state.name
                }
                for t in study.trials
            ],
        },
        f,
        indent=4,
    )

print(f"Optuna results saved to: {file_path}")


In [ ]:
# min_prof_thr, max_down_prop, smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh = stockanalibs.signal_parameters(ticker)


# avg_improvement, improvements = optimiz_function(df=df,
#                                                 min_prof_thr=min_prof_thr, # percent of minimum profit to define a potential trade
#                                                 max_down_prop=max_down_prop, # float (percent/100) of maximum allowed drop of a potential trade
#                                                 smooth_win_sig=smooth_win_sig, # smoothing window of the signal used for the identification of the final trades 
#                                                 pre_entry_decay=pre_entry_decay, # pre-trade decay of the final trades' smoothed signal
#                                                 buy_threshold=buy_threshold, # float (percent/100) threshold of the smoothed signal to trigger the final trade
#                                                 trailing_stop_thresh=trailing_stop_thresh # percent of the trailing stop loss of the final trade
#                                                  )

# print(avg_improvement)
# improvements